<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/notebooks/Testing_finetuned_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls -lh /content/drive/MyDrive/aee4-medgemma-eczema


total 147M
-rw------- 1 root root 1.4K Nov  9 18:09 adapter_config.json
-rw------- 1 root root 147M Nov  9 18:09 adapter_model.safetensors
-rw------- 1 root root 5.2K Nov  9 18:09 README.md


In [3]:
from huggingface_hub import login
login()

In [4]:
!pip install -U transformers peft bitsandbytes accelerate


In [5]:
!pip install unsloth


In [6]:
from unsloth import FastLanguageModel

base_model_name = "unsloth/medgemma-4b-it"
adapter_path = "/content/drive/MyDrive/aee4-medgemma-eczema"

# Load base model first
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    load_in_4bit=True,
)

# Load adapter properly (for new PEFT versions)
from peft import PeftModel

model = PeftModel.from_pretrained(model, adapter_path)
FastLanguageModel.for_inference(model)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (model): Gemma3Model(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(4096, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-15): 16 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): lora.Linear(
                      (base_layer): Linear(in_features=1152, out_features=1152, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.05, inplace=False)
                      )
                      (lor

In [10]:
from unsloth import FastLanguageModel
from transformers import pipeline

# 1️⃣ Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-3-4b",  # change if using another checkpoint
    load_in_4bit=False,               # or True if you want quantization
    dtype="float16"                   # Gemma3 doesn’t support bfloat16/half
)

# 2️⃣ Create the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device="cuda:0"
)

# 3️⃣ Define the system prompt
prompt = (
    "Inside the elbows and behind the knees show patches of skin that are rough and scaly; "

)

# 4️⃣ Generate output (use explicit text=)
output = pipe(
    text=prompt,
    max_new_tokens=120,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

# 5️⃣ Print result
print(output[0]["generated_text"])


RuntimeError: Unsloth: No config file found - are you sure the `model_name` is correct?
If you're using a model on your local device, confirm if the folder location exists.
If you're using a HuggingFace online model, check if it exists.